# Modello di predizione delle emozioni dato del testo

## Data retrival

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/AIExam/train_ekmann.csv /content
!pip install contractions emoji ekphrasis --quiet

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import LSTM, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt
import pandas as pd
import csv
from PIL import Image
import re
import numpy as np
import contractions, emoji
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

EPOCH = 100

In [ ]:
def load_data(file_path):
    data = []
    with open(file_path, newline='', encoding='utf-8') as csvfile:
        csv_reader = csv.reader(csvfile)
        next(csv_reader, None)
        for row in csv_reader:
            text, emotion, id = row
            entry = {'text': text, 'emotion': emotion, 'id': id}
            data.append(entry)
    return data

file_path = "train_ekmann.csv"
data = load_data(file_path)
print(data[0])

{'text': "My favourite food is anything I didn't have to cook myself.", 'emotion': 'neutral', 'id': 'eebbqej'}


In [ ]:
texts = [entry['text'] for entry in data]
emotions = [entry['emotion'] for entry in data]

def show_data(i):
  print("Testo associato:", texts[i])
  print("Emozione:", emotions[i])

show_data(123)

i = 12345
show_data(i)

Testo associato: Three words, no subtlety. "Dude. Stop. Seriously." 
Emozione: anger
Testo associato: Weird, just got back from the one on Grand 45 min ago and I got in and out no problem. 
Emozione: neutral


## Encoding delle emozioni

In [ ]:
label_encoder = LabelEncoder()
encoded_emotions = label_encoder.fit_transform(emotions)
categorical_emotions = tf.keras.utils.to_categorical(encoded_emotions, num_classes=len(label_encoder.classes_))

custom_p = lambda n: print("emotion\t ", n, "  \tis encoded with the number\t", encoded_emotions[emotions.index(n)], "\tand vector\t", categorical_emotions[emotions.index(n)])

custom_p("anger")
custom_p("disgust")
custom_p("fear")
custom_p("joy")
custom_p("neutral")
custom_p("sadness")
custom_p("surprise")

emotion	  anger   	is encoded with the number	 0 	and vector	 [1. 0. 0. 0. 0. 0. 0.]
emotion	  disgust   	is encoded with the number	 1 	and vector	 [0. 1. 0. 0. 0. 0. 0.]
emotion	  fear   	is encoded with the number	 2 	and vector	 [0. 0. 1. 0. 0. 0. 0.]
emotion	  joy   	is encoded with the number	 3 	and vector	 [0. 0. 0. 1. 0. 0. 0.]
emotion	  neutral   	is encoded with the number	 4 	and vector	 [0. 0. 0. 0. 1. 0. 0.]
emotion	  sadness   	is encoded with the number	 5 	and vector	 [0. 0. 0. 0. 0. 1. 0.]
emotion	  surprise   	is encoded with the number	 6 	and vector	 [0. 0. 0. 0. 0. 0. 1.]


## Pre-processing del testo

In [ ]:
max_words = 20000
tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(texts)
text_sequences = tokenizer.texts_to_sequences(texts)
text_padded = pad_sequences(text_sequences, padding='post')

## Creazione del dataset
Dividiamo il dataset nella seguente maniera:
*   train data: 80%  
*   validation data: 10%  
*   test data: 10%  

In [ ]:
text_train, text_test_temp, emotion_train, emotion_test_temp = train_test_split(
    text_padded, categorical_emotions, test_size=0.2, random_state=42
)

text_test, text_val, emotion_test, emotion_val = train_test_split(
    text_test_temp, emotion_test_temp, test_size=0.5, random_state=42
)

print("Lunghezza dati di train:", len(text_train))
print("Lunghezza dati di validation:", len(text_val))
print("Lunghezza dati di test:", len(text_test))

Lunghezza dati di train: 34728
Lunghezza dati di validation: 4341
Lunghezza dati di test: 4341


## Prima prova

In [ ]:
text_input = layers.Input(shape=text_padded.shape[1], dtype=tf.int32)
text_dense_layer = layers.Dense(64, activation='relu')(text_input)

dropout_layer = layers.Dropout(0.5)(text_dense_layer)
output_layer = layers.Dense(len(label_encoder.classes_), activation='softmax')(dropout_layer)

model = models.Model(inputs=text_input, outputs=output_layer)

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy', metrics.Precision(), metrics.Recall()])
model.summary()

checkpoint_callback = ModelCheckpoint("best_model.h5", save_best_only=True, monitor="val_accuracy", mode="max")

early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 142)]             0         
                                                                 
 dense_9 (Dense)             (None, 64)                9152      
                                                                 
 dropout_7 (Dropout)         (None, 64)                0         
                                                                 
 dense_10 (Dense)            (None, 7)                 455       
                                                                 
Total params: 9607 (37.53 KB)
Trainable params: 9607 (37.53 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.fit(text_train, emotion_train, epochs=EPOCH, validation_data=(text_val, emotion_val), callbacks=[checkpoint_callback, early_stopping_callback])

evaluation_results = model.evaluate(text_test, emotion_test)
print(f"Loss: {round(evaluation_results[0], 2)}, Accuracy: {round(evaluation_results[1], 2)}")

Epoch 1/100
1086/1086 [==============================] - 6s 4ms/step - loss: 212.3520 - accuracy: 0.2671 - precision_4: 0.2510 - recall_4: 0.2107 - val_loss: 8.2950 - val_accuracy: 0.3402 - val_precision_4: 0.2891 - val_recall_4: 0.1136
Epoch 2/100
  46/1086 [>.............................] - ETA: 3s - loss: 8.6000 - accuracy: 0.3431 - precision_4: 0.2860 - recall_4: 0.0931

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1086/1086 [==============================] - 6s 6ms/step - loss: 4.6643 - accuracy: 0.3561 - precision_4: 0.2752 - recall_4: 0.0470 - val_loss: 2.2667 - val_accuracy: 0.3628 - val_precision_4: 0.3137 - val_recall_4: 0.0196
Epoch 3/100
1086/1086 [==============================] - 8s 8ms/step - loss: 2.0636 - accuracy: 0.3658 - precision_4: 0.2944 - recall_4: 0.0212 - val_loss: 1.7245 - val_accuracy: 0.3674 - val_precision_4: 0.3318 - val_recall_4: 0.0327
Epoch 4/100
1086/1086 [==============================] - 6s 5ms/step - loss: 1.7160 - accuracy: 0.3696 - precision_4: 0.2725 - recall_4: 0.0092 - val_loss: 1.6672 - val_accuracy: 0.3697 - val_precision_4: 0.3051 - val_recall_4: 0.0124
Epoch 5/100
1086/1086 [==============================] - 4s 4ms/step - loss: 1.6161 - accuracy: 0.3703 - precision_4: 0.2896 - recall_4: 0.0037 - val_loss: 1.5692 - val_accuracy: 0.3709 - val_precision_4: 0.4074 - val_recall_4: 0.0025
Epoch 6/100
1086/1086 [==============================] - 4s 4ms/step - l

Loss: 1.52, Accuracy: 0.39, Precision: 0.33, Recall: 0.0002.

## Seconda prova

In [ ]:
text_input = layers.Input(shape=text_padded.shape[1], dtype=tf.int32)
embedding_layer = layers.Embedding(input_dim=max_words, output_dim=8, input_length=text_padded.shape[1])(text_input)
text_flatten = layers.Flatten()(embedding_layer)

dropout_layer = layers.Dropout(0.5)(text_flatten)
output_layer = layers.Dense(len(label_encoder.classes_), activation='softmax')(dropout_layer)

model = models.Model(inputs=text_input, outputs=output_layer)
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy', metrics.Precision(), metrics.Recall()])
model.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 142)]             0         
                                                                 
 embedding_5 (Embedding)     (None, 142, 8)            160000    
                                                                 
 flatten_5 (Flatten)         (None, 1136)              0         
                                                                 
 dropout_8 (Dropout)         (None, 1136)              0         
                                                                 
 dense_11 (Dense)            (None, 7)                 7959      
                                                                 
Total params: 167959 (656.09 KB)
Trainable params: 167959 (656.09 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.fit(text_train, emotion_train, epochs=EPOCH, validation_data=(text_val, emotion_val), callbacks=[checkpoint_callback, early_stopping_callback])

evaluation_results = model.evaluate(text_test, emotion_test)
print(f"Loss: {round(evaluation_results[0], 2)}, Accuracy: {round(evaluation_results[1], 2)}")

Epoch 1/100
1086/1086 [==============================] - 20s 17ms/step - loss: 1.5820 - accuracy: 0.3630 - precision_5: 0.0000e+00 - recall_5: 0.0000e+00 - val_loss: 1.5343 - val_accuracy: 0.3709 - val_precision_5: 0.0000e+00 - val_recall_5: 0.0000e+00
Epoch 2/100
1086/1086 [==============================] - 7s 6ms/step - loss: 1.5297 - accuracy: 0.3722 - precision_5: 0.0000e+00 - recall_5: 0.0000e+00 - val_loss: 1.5294 - val_accuracy: 0.3709 - val_precision_5: 0.0000e+00 - val_recall_5: 0.0000e+00
Epoch 3/100
1086/1086 [==============================] - 5s 5ms/step - loss: 1.5214 - accuracy: 0.3774 - precision_5: 0.9333 - recall_5: 4.0313e-04 - val_loss: 1.5186 - val_accuracy: 0.3709 - val_precision_5: 0.0000e+00 - val_recall_5: 0.0000e+00
Epoch 4/100
1086/1086 [==============================] - 6s 5ms/step - loss: 1.5057 - accuracy: 0.3961 - precision_5: 0.9295 - recall_5: 0.0084 - val_loss: 1.5005 - val_accuracy: 0.3928 - val_precision_5: 0.9859 - val_recall_5: 0.0161
Epoch 5/100
10

Loss: 1.13, Accuracy: 0.6, Precision: 0.71, Recall: 0.43.

## Terza prova

In [ ]:
def lowerize(data):
  return list(map(lambda x: x.lower(), data))

def remove_special_chars(data):
  return list(map(lambda x: x.replace('|'.join([re.escape(c) for c in list("#%&*/:\^_{|}~")]), ""), data))

def convert_emojis(data):
  return list(map(lambda x: emoji.demojize(x), data))

def split_contractions(data):
  return list(map(lambda x: contractions.fix(x), data))

text_processor = TextPreProcessor(normalize= ['money', 'user', 'time', 'date', 'number', 'phone'],
                                  annotate={"elongated", "repeated", 'emphasis', 'censored'},
                                  fix_html=True, tokenizer=SocialTokenizer(lowercase=True).tokenize,
                                  segmenter="twitter", corrector="twitter", unpack_contractions=False,
                                  spell_correct_elong=True, spell_correction=True, fix_text=True,
                                  dicts=[emoticons])

def df_preprocessing(df_pre):
  df_pre = lowerize(df_pre)
  df_pre = convert_emojis(df_pre)
  df_pre = split_contractions(df_pre)
  df_pre = list(map(lambda x: ' '.join(text_processor.pre_process_doc(x)).strip(), df_pre))
  df_pre = remove_special_chars(df_pre)
  return df_pre

Reading twitter - 1grams ...


In [ ]:
texts = [entry['text'] for entry in data]
emotions = [entry['emotion'] for entry in data]

texts = df_preprocessing(texts)

max_words = 20000
tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(texts)
text_sequences = tokenizer.texts_to_sequences(texts)
text_padded = pad_sequences(text_sequences, padding='post')
label_encoder = LabelEncoder()
encoded_emotions = label_encoder.fit_transform(emotions)
categorical_emotions = tf.keras.utils.to_categorical(encoded_emotions, num_classes=len(label_encoder.classes_))

text_train, text_test_temp, emotion_train, emotion_test_temp = train_test_split(
    text_padded, categorical_emotions, test_size=0.2, random_state=42
)

text_test, text_val, emotion_test, emotion_val = train_test_split(
    text_test_temp, emotion_test_temp, test_size=0.2, random_state=42
)

In [ ]:
text_input = layers.Input(shape=text_padded.shape[1], dtype=tf.int32)
embedding_layer = layers.Embedding(input_dim=max_words, output_dim=8, input_length=text_padded.shape[1])(text_input)
text_flatten = layers.Flatten()(embedding_layer)

dropout_layer = layers.Dropout(0.5)(text_flatten)
output_layer = layers.Dense(len(label_encoder.classes_), activation='softmax')(dropout_layer)

optimizer = Adam(learning_rate=0.0001)
model = models.Model(inputs=text_input, outputs=output_layer)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy', metrics.Precision(), metrics.Recall()])
model.summary()

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 485)]             0         
                                                                 
 embedding_7 (Embedding)     (None, 485, 8)            160000    
                                                                 
 flatten_7 (Flatten)         (None, 3880)              0         
                                                                 
 dropout_10 (Dropout)        (None, 3880)              0         
                                                                 
 dense_13 (Dense)            (None, 7)                 27167     
                                                                 
Total params: 187167 (731.12 KB)
Trainable params: 187167 (731.12 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.fit(text_train, emotion_train, epochs=EPOCH, validation_data=(text_val, emotion_val), callbacks=[checkpoint_callback, early_stopping_callback])

evaluation_results = model.evaluate(text_test, emotion_test)
print(f"Loss: {round(evaluation_results[0], 2)}, Accuracy: {round(evaluation_results[1], 2)}")

Epoch 1/100
1086/1086 [==============================] - 15s 13ms/step - loss: 1.5560 - accuracy: 0.3679 - precision_7: 0.0000e+00 - recall_7: 0.0000e+00 - val_loss: 1.5235 - val_accuracy: 0.3673 - val_precision_7: 0.0000e+00 - val_recall_7: 0.0000e+00
Epoch 2/100
1086/1086 [==============================] - 6s 5ms/step - loss: 1.5273 - accuracy: 0.3747 - precision_7: 0.0000e+00 - recall_7: 0.0000e+00 - val_loss: 1.5152 - val_accuracy: 0.3673 - val_precision_7: 0.0000e+00 - val_recall_7: 0.0000e+00
Epoch 3/100
1086/1086 [==============================] - 5s 5ms/step - loss: 1.5123 - accuracy: 0.3843 - precision_7: 1.0000 - recall_7: 5.7590e-04 - val_loss: 1.4913 - val_accuracy: 0.3955 - val_precision_7: 1.0000 - val_recall_7: 5.7571e-04
Epoch 4/100
1086/1086 [==============================] - 5s 5ms/step - loss: 1.4846 - accuracy: 0.4252 - precision_7: 0.9317 - recall_7: 0.0251 - val_loss: 1.4608 - val_accuracy: 0.4398 - val_precision_7: 0.9577 - val_recall_7: 0.0391
Epoch 5/100
1086/1

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1086/1086 [==============================] - 5s 4ms/step - loss: 1.1161 - accuracy: 0.6073 - precision_7: 0.7914 - recall_7: 0.3977 - val_loss: 1.1470 - val_accuracy: 0.5936 - val_precision_7: 0.7628 - val_recall_7: 0.3610
Epoch 31/100
1086/1086 [==============================] - 4s 4ms/step - loss: 1.1109 - accuracy: 0.6068 - precision_7: 0.7849 - recall_7: 0.3989 - val_loss: 1.1448 - val_accuracy: 0.5924 - val_precision_7: 0.7438 - val_recall_7: 0.3794
Epoch 32/100
1086/1086 [==============================] - 5s 5ms/step - loss: 1.1015 - accuracy: 0.6124 - precision_7: 0.7912 - recall_7: 0.4079 - val_loss: 1.1424 - val_accuracy: 0.5953 - val_precision_7: 0.7557 - val_recall_7: 0.3794
Epoch 33/100
1086/1086 [==============================] - 4s 4ms/step - loss: 1.1014 - accuracy: 0.6115 - precision_7: 0.7866 - recall_7: 0.4084 - val_loss: 1.1391 - val_accuracy: 0.5918 - val_precision_7: 0.7461 - val_recall_7: 0.3857
Epoch 34/100
1086/1086 [==============================] - 5s 5ms/step

Loss: 1.12, Accuracy: 0.6, Precision: 0.72, Recall: 0.4.